In [1]:
import json
import pandas as pd
import firebase_admin
from datetime import date
from datetime import datetime
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:

df = pd.read_csv('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Text Normalization\\Tweets.csv')

In [3]:
df.dropna(inplace= True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4791 entries, 0 to 4790
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4791 non-null   int64 
 1   Tweet       4791 non-null   object
 2   Location    4791 non-null   object
 3   year        4791 non-null   int64 
 4   month       4791 non-null   int64 
 5   day         4791 non-null   int64 
 6   Sentiment   4791 non-null   object
dtypes: int64(4), object(3)
memory usage: 299.4+ KB


In [5]:
CredNew = credentials.Certificate('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Database\\smartcc-960f7-firebase-adminsdk-ijo0i-27aa89d85e.json')
new=firebase_admin.initialize_app(CredNew,name="new")

db = firestore.client(new)

In [6]:
csv= json.loads(json.dumps(df[0:2000].to_dict()))
csv2= json.loads(json.dumps(df[2000:].to_dict()))

In [8]:


# datetime object containing current date and time

today = date.today()
today= str(today)

today1 = date.today()
today1= str(today1)


In [9]:
tweets= db.collection(u'test').document(today).set(csv)
tweets1= db.collection(u'test').document(today1).set(csv2)

# will use these commented codes in upcoming phases # 

In [ ]:
'''
DataOld=[]
ids=[]
all_users_ref_2 = db.collection(u'test').stream()
for users in all_users_ref_2:
    #print(u'{} => {}'.format(users.id, users.to_dict()))
    DataOld.append(users.to_dict())
    ids.append(users.id)
'''

In [ ]:
'''
dataNew=[]
idsNew=[]
all_users_ref_2 = db.collection(u'test').stream()
for users in all_users_ref_2:
    #print(u'{} => {}'.format(users.id, users.to_dict()))
    dataNew.append(users.to_dict())
    idsNew.append(users.id)
'''

In [ ]:
#compare between new data and old one
'''
for i in range(len(DataOld)):
    if DataOld[i] not in dataNew:
        db.collection(u'test').document(ids[i]).set(DataOld[i])
    else:
        print("data already exists")
    '''

In [ ]:
#read old data from old firebase account
'''
# Use a service account
CredOld = credentials.Certificate('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Database\\smartcc-26e57-85547b8418c0.json')
old=firebase_admin.initialize_app(CredOld,name="old")

db = firestore.client(old)
'''

In [ ]:
# read full collection in the new account
'''

database_2 = firestore.client()
all_users_ref_2 = database_2.collection(u'test').stream()
for users in all_users_ref_2:
  print(u'{} => {}'.format(users.id, users.to_dict()))

'''

In [ ]:
'''

from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
now = str(now) 

now1 = datetime.now()
now1 = str(now1) 
'''